### Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from pathlib import Path
scripts_path = Path("../Data-Preprocessing/").resolve()
sys.path.append(str(scripts_path))
import os
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'

In [3]:
import pickle
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestCentroid
from scripts.data_visualiser import *
from scripts.data_loader import *
from scripts.data_preprocessor import *
from scripts.temporal_data_preprocessor import *
from scripts.temporal_data_loader import *
from scripts.temporal_visualiser import *
from scripts.temporal_chanel_refinement import *
from model_scripts.get_statistics import *
from model_scripts.dataset_creation import *
from model_scripts.train_model_ae import *
from model_scripts.model_visualiser import *
from model_scripts.superpixel import *
from model_scripts.pre_trained_extraction import *
from Pipeline.pre_processing_pipeline import *
from Pipeline.temporal_preprocessing_pipeline import *
import numpy as np
import preprocessing_config as config
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
import skimage.measure
import torch
import torch.nn as nn
import torch.optim as optim
from torchgeo.models import resnet18
from torchgeo.models import ResNet18_Weights
from sklearn.cluster import KMeans

### Loading the data

In [4]:
# temp_pipeline = PreProcessingPipelineTemporal()

In [5]:
# train_fn, dataloader_train = temp_pipeline.get_processed_trainloader(64, 'indexbands', vi_type='msi')

Train

In [6]:
temporal_images = load_field_images_temporal(config.base_directory_temporal_train)
border_removed_images_train = blacken_field_borders_temporal(temporal_images)
field_numbers_train, indices_images_train = allbands_temporal_cubes(border_removed_images_train)

len(indices_images_train), indices_images_train[0][0].shape

(2425, (64, 64, 10))

Test

In [7]:
temporal_images_test = load_field_images_temporal(config.base_directory_temporal_test)
border_removed_images = blacken_field_borders_temporal(temporal_images_test)
field_numbers_test, indices_images_test = allbands_temporal_cubes(border_removed_images)

(len(indices_images_test), indices_images_test[0][0].shape)

(48, (64, 64, 10))

In [8]:
image_tensor_train = np.stack(indices_images_train)  # Shape: (N x 7 x 64 x 64 x 6)
image_tensor_test = np.stack(indices_images_test)   # Shape: (N x 7 x 64 x 64 x 6)

image_tensor_train.shape, image_tensor_test.shape

((2425, 7, 64, 64, 10), (48, 7, 64, 64, 10))

### Mini-patches

In [9]:
image_tensor_train = torch.tensor(image_tensor_train, dtype=torch.float32).permute(0, 1, 4, 2, 3)  # (N, T, H, W, C) -> (N, T, C, H, W)
image_tensor_test = torch.tensor(image_tensor_test, dtype=torch.float32).permute(0, 1, 4, 2, 3)  # (N, T, H, W, C) -> (N, T, C, H, W)
image_tensor_train.shape, image_tensor_test.shape

(torch.Size([2425, 7, 10, 64, 64]), torch.Size([48, 7, 10, 64, 64]))

In [10]:
train_patches, train_patch_coordinates = non_overlapping_sliding_window(image_tensor_train, field_numbers_train, patch_size=4)
test_patches, test_patch_coordinates = non_overlapping_sliding_window(image_tensor_test, field_numbers_test, patch_size=4)
len(test_patches), test_patches[0].shape

(1197, torch.Size([7, 10, 4, 4]))

### Single Timestamp Images

In [11]:
train_patches_tensor = torch.stack(train_patches)  # Convert to tensor [N, T, C, H, W]
test_patches_tensor = torch.stack(test_patches)

In [12]:
train_last_timestep_patches = get_last_timestep_patches(train_patches_tensor)
test_last_timestep_patches = get_last_timestep_patches(test_patches_tensor)

test_last_timestep_patches.shape

torch.Size([1197, 10, 4, 4])

### k-means

In [ ]:
kmeans = train_kmeans_patches(train_last_timestep_patches, n_clusters=2, random_state=9)

train_patch_predictions = kmeans.predict(train_last_timestep_patches.reshape(train_last_timestep_patches.size(0), -1).numpy())
test_patch_predictions = kmeans.predict(test_last_timestep_patches.reshape(test_last_timestep_patches.size(0), -1).numpy())

# Assign field labels
threshold = 4
train_field_labels = assign_field_labels(train_patch_coordinates, train_patch_predictions, threshold)
test_field_labels = assign_field_labels(test_patch_coordinates, test_patch_predictions, threshold)

In [ ]:
accuracy, report = evaluate_test_labels(test_field_labels, config.labels_path)
print(f"Test Accuracy: {accuracy}")
print(report)

## Pre-trained Models

In [ ]:
test_last_timestep_patches.shape

#### RESNET18 - ALL Bands

In [ ]:
# train_features = extract_features_resnet(dataset=train_last_timestep_patches, num_channels=10, pretrained_weights=ResNet18_Weights.SENTINEL2_ALL_MOCO)
# train_features = extract_features_resnet(dataset=test_last_timestep_patches, num_channels=10, pretrained_weights=ResNet18_Weights.SENTINEL2_ALL_MOCO)

In [ ]:
train_features = extract_features_resnet(dataset=train_last_timestep_patches, num_channels=10, pretrained_weights=ResNet18_Weights.SENTINEL2_ALL_MOCO)
test_features = extract_features_resnet(dataset=test_last_timestep_patches, num_channels=10, pretrained_weights=ResNet18_Weights.SENTINEL2_ALL_MOCO)

In [ ]:
kmeans = train_kmeans_patches(train_features, n_clusters=2, random_state=16)

train_patch_predictions = kmeans.predict(train_features.reshape(train_features.size(0), -1).numpy())
test_patch_predictions = kmeans.predict(test_features.reshape(test_features.size(0), -1).numpy())

# Assign field labels
threshold = 4
train_field_labels = assign_field_labels(train_patch_coordinates, train_patch_predictions, threshold)
test_field_labels = assign_field_labels(test_patch_coordinates, test_patch_predictions, threshold)

In [ ]:
accuracy, report = evaluate_test_labels(test_field_labels, config.labels_path)
print(f"Test Accuracy: {accuracy}")
print(report)

#### RESNET18 - RGB Bands

In [13]:
train_features = extract_features_resnet_rgb(dataset=train_last_timestep_patches, num_channels=3, pretrained_weights=ResNet18_Weights.SENTINEL2_RGB_MOCO)
test_features = extract_features_resnet_rgb(dataset=test_last_timestep_patches, num_channels=3, pretrained_weights=ResNet18_Weights.SENTINEL2_RGB_MOCO)

In [14]:
kmeans = train_kmeans_patches(train_features, n_clusters=2, random_state=20)

train_patch_predictions = kmeans.predict(train_features.reshape(train_features.size(0), -1).numpy())
test_patch_predictions = kmeans.predict(test_features.reshape(test_features.size(0), -1).numpy())

# Assign field labels
threshold = 4
train_field_labels = assign_field_labels(train_patch_coordinates, train_patch_predictions, threshold)
test_field_labels = assign_field_labels(test_patch_coordinates, test_patch_predictions, threshold)

/Users/bhumikasadbhave007/tensorflow-test/env/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [15]:
accuracy, report = evaluate_test_labels(test_field_labels, config.labels_path)
print(f"Test Accuracy: {accuracy}")
print(report)

Test Accuracy: 0.5409836065573771
              precision    recall  f1-score   support

           0       0.33      0.08      0.12        26
           1       0.56      0.89      0.69        35

    accuracy                           0.54        61
   macro avg       0.45      0.48      0.41        61
weighted avg       0.47      0.54      0.45        61



: 